In [26]:
!pip install pandas
!pip install numpy
import pandas as pd
import numpy as np
import re

In [9]:
pd.set_option("display.max_rows", None)

In [10]:
df = pd.read_csv('mtDNATree.csv')

In [11]:
df.head()

,mt-MRCA (RSRS),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L0,G263A C1048T C3516a T5442C T6185C C9042T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,L0a'b'f'g'k,A189G T4586C C9818T T16172C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def isHaplotype(df, row, col):
    """
    A cell is a haplotype if it is not NaN,
    its left is NaN, and its right is a string.
    """
    if pd.isna(df.iloc[row, col]):
        return False

    left_cell = df.iloc[row, col - 1] if col > 0 else np.nan
    if not pd.isna(left_cell):
        return False

    right_cell = df.iloc[row, col + 1] if col < len(df.columns) - 1 else np.nan
    if not isinstance(right_cell, str):
        return False

    return True


In [18]:
def isBranchMutation(df, row, col):
    """
    A cell is a branch mutation if:
    - It contains a string.
    - Both its left and right neighbors are NaN.
    """
    if not isinstance(df.iloc[row, col], str):
        return False

    left_cell = df.iloc[row, col - 1] if col > 0 else np.nan
    if not pd.isna(left_cell):
        return False

    right_cell = df.iloc[row, col + 1] if col < len(df.columns) - 1 else np.nan
    if not pd.isna(right_cell):
        return False

    return True


In [87]:
haplogroup_and_mutations = {}
branchMutationsPerColumn = {}

ROWS, COLUMNS = len(df), len(df.columns)

for col_idx in range(COLUMNS):
    branchMutationsPerColumn[col_idx] = None # start with no branch butation 

for col_idx in range(COLUMNS):
    for row in range(ROWS):
        cell = df.iloc[row, col_idx]

        if isBranchMutation(df, row, col_idx):
            # reset the branch mutations for this column, because we got a new branch mutation for on this level
            branchMutationsPerColumn[col_idx] = cell
        elif isHaplotype(df, row, col_idx):
            branchMutations = [branchMutationsPerColumn[i] for i in range(col_idx + 1) if branchMutationsPerColumn[i]]

            # Get the mutations from the cell to the right
            if col_idx + 1 < len(df.columns):
                mutations_cell = df.iloc[row, col_idx + 1]
                if pd.isna(mutations_cell):
                    mutations_cell = ''
            else:
                mutations_cell = ''

            mutations_list = []

            if mutations_cell:
                mutations_list.append(mutations_cell)

            mutations_list.extend(branchMutations)

            total_mutations = ' '.join(mutations_list).strip()

            haplogroup_and_mutations[cell] = total_mutations

        else:
            continue


In [88]:
df_haplogroup_and_mutations = pd.DataFrame(
    list(haplogroup_and_mutations.items()), # convert the dict into a list of tuples to make the df
    columns=["Haplogroup", "Mutations"]
)

In [89]:
df_haplogroup_and_mutations.head()

,Haplogroup,Mutations
0,L0,G263A C1048T C3516a T5442C T6185C C9042T ...
1,L1'2'3'4'5'6,C146T C182T T4312C T10664C C10915T A11914...
2,L0a'b'f'g'k,A189G T4586C C9818T T16172C
3,L0d,G1438A T4232C T6815C C8113a G8152A G8251A...
4,L1,G3666A A7055G T7389C T13789C T14178C G14560A


In [90]:
df_haplogroup_and_mutations.count()

Haplogroup    8762
Mutations     8762
dtype: int64

In [91]:
def classify_mutation(mutation):
    """
    Matches the mutation to its pattern 
    to classifiy the type of the mutation
    """
    if re.match(r"^[ACGT]\d+[ACGT]$", mutation):
        if mutation[-1].islower():
            return "Transversion"
        else:
            return "Transition"
    elif re.match(r"^\d+\.\d+[ACGT]+$", mutation):
        return "Insertion"
    elif re.match(r"^[ACGT]\d+d$", mutation):
        return "Deletion"
    elif "!" in mutation:
        return "Back Mutation"
    elif re.match(r"^\(\d+[ACGT]+\)$", mutation):
        return "Recurrent/Unstable"
    else:
        return "Unknown"

In [92]:
individual_mutations = []

for mutations in df_haplogroup_and_mutations["Mutations"]:
    """
    go through each mutation, and split it on space
    to get a list of mutations, mark all of them 
    for the next dataframe.
    """
    for mutation in mutations.split():
        mutation_type = classify_mutation(mutation)
        individual_mutations.append({"Mutation": mutation, "Type": mutation_type})

In [93]:
mutation_df = pd.DataFrame(individual_mutations) # create dataframe out of the list
mutation_df["Frequency"] = mutation_df.groupby("Mutation")["Mutation"].transform("count") # group by the mutation to remove duplicates and keep the count
mutation_df = mutation_df.drop_duplicates() # drop any duplicates, should not be any

In [94]:
mutation_df.head()

,Mutation,Type,Frequency
0,G263A,Transition,13
1,C1048T,Transition,4
2,C3516a,Unknown,1
3,T5442C,Transition,6
4,T6185C,Transition,4


In [95]:
import sqlite3

connection = sqlite3.connect("haplogroup_mutations.db")

# save the haplogroup dataframe to the database as the haplogroup_data table
df_haplogroup_and_mutations.to_sql("haplogroup_data", connection, if_exists="replace", index=False)
# save the mutation dataframe to the database as mutation_data table
mutation_df.to_sql("mutation_data", connection, if_exists="replace", index=False)

connection.commit()
connection.close()

print("Saved dataframes to database!")

Saved dataframes to database!


In [96]:
import sqlite3

# connect to database
connection = sqlite3.connect("haplogroup_mutations.db")

# Create a cursor for executing queries
cursor = connection.cursor()

# Get a list of all haplogrups
cursor.execute("SELECT DISTINCT Haplogroup FROM haplogroup_data")
haplogroups = cursor.fetchall()
print("List of all haplogroups:")
print([row[0] for row in haplogroups])

# Get list of all mutations
cursor.execute("SELECT DISTINCT Mutation FROM mutation_data")
mutations = cursor.fetchall()
print("\nList of all mutations:")
print([row[0] for row in mutations])

# Find mutations for a specific haplogroup
specific_haplogroup = "L0"  # Replace with desired haplogroup
cursor.execute("""
    SELECT m.Mutation, m.Type, m.Frequency
    FROM haplogroup_data h
    JOIN mutation_data m ON h.Mutations LIKE '%' || m.Mutation || '%'
    WHERE h.Haplogroup = ?
""", (specific_haplogroup,))
haplogroup_mutations = cursor.fetchall()
print(f"\nMutations for haplogroup '{specific_haplogroup}':")
print(haplogroup_mutations)

# 4. Haplogroup with the most defining mutations
cursor.execute("""
    SELECT Haplogroup, COUNT(*) AS mutation_count
    FROM haplogroup_data
    GROUP BY Haplogroup
    ORDER BY mutation_count DESC
    LIMIT 1
""")
most_mutations = cursor.fetchone()
print("\nHaplogroup with the most defining mutations:")
print(most_mutations)

# 5. Haplogroups with a specific mutation
specific_mutation = "G263A"  # replace with any other mutation
cursor.execute("""
    SELECT DISTINCT h.Haplogroup
    FROM haplogroup_data h
    JOIN mutation_data m ON h.Mutations LIKE '%' || m.Mutation || '%'
    WHERE m.Mutation = ?
""", (specific_mutation,))
haplogroups_with_mutation = cursor.fetchall()
print(f"\nHaplogroups containing the mutation '{specific_mutation}':")
print([row[0] for row in haplogroups_with_mutation])

connection.close()

List of all haplogroups:
['L0', "L1'2'3'4'5'6", "L0a'b'f'g'k", 'L0d', 'L1', "L2'3'4'5'6", "L0a'b'f'g", 'L0k', "L0d1'2", 'L0d3', 'L1b', 'L1c', 'L5', "L2'3'4'6", "L0a'b'g", 'L0f', 'L0k1', 'L0k2', 'L0d1', 'L0d2', 'L0d3a', 'L0d3b', 'L1b1', "L1b2'3", "L1c1'2'4'5'6", 'L1c3', 'L5a', 'L5b', 'L2', "L3'4'6", "L0a'g", 'L0b', 'L0f1', 'L0f2', 'L0k1a', 'L0k1b', 'L0k2a', 'L0k2b', "L0d1a'c'd", 'L0d1b', "L0d2a'b'd", 'L0d2c', 'L0d3b1', 'L0d3b2', 'L1b1a', 'L1b2', 'L1b3', "L1c1'2'4'6", 'L1c5', 'L1c3a', "L1c3b'c", 'L5a1', 'L5a2', 'L5b1', 'L5b2', "L2a'b'c'd", 'L2e', 'L6', "L3'4", 'L0a', 'L0g', 'L0f2a', 'L0f2b', 'L0k1a1', 'L0k1a2', 'L0k1a3', 'L0k2a1', "L0d1a'd", 'L0d1c', 'L0d1b1', 'L0d1b2', 'L0d2a', 'L0d2b', 'L0d2d', 'L0d2c1', 'L0d2c2', "L1b1a1'4", 'L1b1a2', 'L1b1a5', 'L1b1a6', 'L1b1a7', 'L1b1a8', 'L1b1a10', 'L1b1a12', 'L1b1a13', 'L1b1a14', 'L1b1a16', 'L1b2a', 'L1c1', "L1c2'4", 'L1c6', 'L1c3a1', 'L1c3b', 'L1c3c', 'L5a1a', 'L5a1b', 'L5a1c', 'L5b1a', 'L5b1b', 'L2a', "L2b'c'd", 'L2e1', 'L6a', 'L6b', 'L4', 'L3',